<a href="https://colab.research.google.com/github/Zhu-Pengming/Flora-Talks/blob/main/flora_talks_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安装 tflite-model-maker 之前的其它依赖
!pip install scann==1.3.1

# 安装 tflite-model-maker
!pip install tflite-model-maker==0.4.3

# 卸载 tflite-support-nightly
!pip uninstall -y tflite-support-nightly

# 重新安装 tflite-support-nightly
!pip install tflite-support-nightly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 37.7 MB/s eta 0:00:00
  Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 72.3 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: protobuf
  

In [ ]:
!apt-get install -y libportaudio2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libportaudio2 is already the newest version (19.6.0-1.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import numpy as np
import os

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.text_classifier import AverageWordVecSpec
from tflite_model_maker.text_classifier import DataLoader

from tflite_support.task import core
from tflite_support.task import processor
from tflite_support.task import text

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

ModuleNotFoundError: No module named 'tflite_model_maker'

In [ ]:
from google.colab import drive
import pandas as pd
import os
from tflite_model_maker import text_classifier, DataLoader

# 挂载 Google Drive
drive.mount('/content/drive')

# 文件路径
data_dir = '/content/drive/My Drive/Flora'

def replace_label(original_file, new_file):
    # Load the original file to pandas. Since the dataset is in Excel format,
    # we use pd.read_excel instead of pd.read_csv
    df = pd.read_excel(original_file)
    # Define how we want to change the label name
    label_map = {0: 'rust', 1: 'powder'}

    # Execute the label change
    df.replace({'label': label_map}, inplace=True)

    # Write the updated dataset to a new file
    df.to_csv(new_file, index=False)

# Replace the label name for both the powder and rust dataset. Then write the
# updated CSV dataset to the current folder.
replace_label(os.path.join(data_dir, 'cleaned_powder.xlsx'), os.path.join(data_dir, 'cleaned_powder.csv'))
replace_label(os.path.join(data_dir, 'cleaned_rust.xlsx'), os.path.join(data_dir, 'cleaned_rust.csv'))

In [ ]:
# Define the model specification
spec = text_classifier.AverageWordVecModelSpec()

In [ ]:
# Load the data
train_data = DataLoader.from_csv(
filename=os.path.join(data_dir, 'cleaned_powder.csv'),
text_column='text',
label_column='label',
model_spec=spec,
is_training=True)
test_data = DataLoader.from_csv(
filename=os.path.join(data_dir, 'cleaned_rust.csv'),
text_column='text',
label_column='label',
model_spec=spec,
is_training=False)

In [ ]:
model = text_classifier.create(train_data, model_spec=spec, epochs=10)

In [ ]:
model.export(export_dir='average_word_vec')